Import necessary modules

In [67]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score
import pprint


In [68]:
# Load the dataset
dataset = pd.read_csv('datasets/phonation.csv')


In [69]:
dataset["class"] = dataset["source_file"].str[:4]
label_encoder = LabelEncoder()
dataset['class'] = label_encoder.fit_transform(dataset['class'])

In [70]:
# Drop rows with null values
dataset = dataset.dropna()

In [71]:
# Drop the 'source_file' column
dataset = dataset.drop(columns=['source_file'])

In [72]:
print(dataset)

      avg DF0  avg DDF0  avg Jitter  avg Shimmer    avg apq   avg ppq  \
0   -0.689701 -0.154107    5.186153     8.595405  26.417864  6.123561   
1   -0.452383 -0.054730    3.280561     7.292266  11.901667  2.758843   
2   -0.300312 -0.041383    3.915843    13.000368  19.847925  3.823546   
3   -0.327381  0.143272    3.817988     4.178263   4.926572  3.782958   
4   -0.412879 -0.401515    4.418473     6.818965  10.745099  3.807577   
..        ...       ...         ...          ...        ...       ...   
903 -0.734472 -0.172394    4.645327    13.935182  28.065810  4.234599   
904 -0.192419  0.280107    4.594869     6.775348  25.966223  4.771735   
905 -0.058561  0.055881    4.587401     8.632771  37.102908  5.218027   
906 -1.820827  0.122169    5.122788     8.669970  30.163675  6.098207   
907  0.636622  0.733071    4.538438     7.607408  32.384390  4.629289   

      avg logE    std DF0   std DDF0  std Jitter  ...  skewness ppq  \
0   -16.512226  35.304337  57.987297    8.968350  ..

In [73]:
# Separate features and target variable
X = dataset.drop(columns=['class'])
y = dataset['class']
print(X.shape)

(904, 28)


In [74]:
# Normalize the feature data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

In [75]:
print(X_normalized.shape)

(904, 28)


In [76]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [77]:
# Initialize and train the Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)


In [78]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf_model, X_normalized, y, cv=kf, scoring='accuracy')

In [79]:
pprint.pp(scores)
print("Mean Accuracy:", scores.mean())

array([0.88950276, 0.91712707, 0.90055249, 0.91160221, 0.92777778])
Mean Accuracy: 0.9093124616329037
